In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=GteCHyOhXXt3dmyixHNVxZS-p2RVfzsNChW164KL65c&tc=t44bWpTCgdULb50IJHJTPB_8GmbzJSNloqt8Htx0I_I&cc=lWkDdsDq6dHwtQtWv0UZ006k4YGETXIqhp4TWWw6f2Q

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJr40hdojgwESrUDhtx43JWsqZ8oaPQFPJxY1U-IU6yt1Aur2taNqUs

Successfully saved authorization token.


In [ ]:
# export meterological data from ERA5
import pprint 
from IPython.display import Image
meter=ee.ImageCollection("ECMWF/ERA5/DAILY")
border=ee.FeatureCollection('users/rmohon835/NW_border')
border=border.geometry()
#pprint.pprint(border.getInfo())

#pprint.pprint(type(border))

pheno2005=meter.filterBounds(border)

precp1=pheno2005.filterDate('2005','2006').select('maximum_2m_air_temperature')
# url=precp1.getThumbUrl({
#     'min':0,'max':0.02,
#     'dimensions': 500, 'region': border,
#     'palette': ['blue', 'yellow', 'orange', 'red']
# })
# Image(url=url)
print(precp1.getInfo())

def imageCollectionToDrive(collection,folder,region):
    imageList = collection.toList(collection.size())
    for n in range(0,collection.size().getInfo()):
        image = ee.Image(imageList.get(n)).clip(border)
        name = image.get("system:index").getInfo()
        task = ee.batch.Export.image.toDrive(image=image,description=name,folder=folder,region=region,maxPixels=int(1e13))
        task.start()
        print("exporting {} to folder '{}' in Google Drive".format(name, folder))

imageCollectionToDrive(precp1,'maxAirTemp2005',border)

In [ ]:
# export year 2005 pheno data 
border=ee.FeatureCollection('users/rmohon835/NW_border').geometry()
pheno=ee.ImageCollection("MODIS/006/MCD12Q2")
cornArea=ee.Image('projects/ee-rmohon835/assets/cornArea/NDVImax2005')

pheno2005=pheno.filterBounds(border).filterDate('2005').first().clip(border);

ls=['Greenup_1','MidGreenup_1','Peak_1','Maturity_1','Senescence_1','MidGreendown_1','Dormancy_1']

for i in range(len(ls)):
  stageName=ls[i]
  image=pheno2005.select(stageName).mask(cornArea).clip(border)
  task=ee.batch.Export.image.toDrive(
      image=image,
      description='2005'+stageName,
      folder='2005newPheno',
      region=border,
      crs='EPSG:4326',
      scale=1000,
      maxPixels=int(1e13))
  task.start()
  print(stageName)



Greenup_1
MidGreenup_1
Peak_1
Maturity_1
Senescence_1
MidGreendown_1
Dormancy_1


In [ ]:
# export all year's phenology data
border=ee.FeatureCollection('users/rmohon835/NW_border').geometry()
pheno=ee.ImageCollection("MODIS/006/MCD12Q2")
cornArea=ee.Image('projects/ee-rmohon835/assets/cornArea/NDVImax2005')

ls=['Greenup_1','MidGreenup_1','Peak_1','Maturity_1','Senescence_1','MidGreendown_1','Dormancy_1']

for year in range(2004,2020):
  cornArea=ee.Image('projects/ee-rmohon835/assets/cornArea/NDVImax'+str(year))
  phenoData=pheno.filterBounds(border).filterDate(str(year)).first().clip(border)
  for i in range(len(ls)):
    stageName=ls[i]
    outName=str(year)+stageName
    image=phenoData.select(stageName).mask(cornArea).clip(border)
    task=ee.batch.Export.image.toDrive(
      image=image,
      description=outName,
      folder='PhenoAllYears',
      region=border,
      crs='EPSG:4326',
      scale=1000,
      maxPixels=int(1e13))
    task.start()
    print(outName)

2017Dormancy_1
2018Greenup_1
2018MidGreenup_1
2018Peak_1
2018Maturity_1
2018Senescence_1
2018MidGreendown_1
2018Dormancy_1
2019Greenup_1
2019MidGreenup_1
2019Peak_1
2019Maturity_1
2019Senescence_1
2019MidGreendown_1
2019Dormancy_1


In [ ]:
border=ee.FeatureCollection('users/rmohon835/NW_border').geometry()
pheno=ee.ImageCollection("MODIS/006/MCD12Q2")
cornArea=ee.Image('projects/ee-rmohon835/assets/cornArea/NDVImax2005')


# export all year's phenology data
border=ee.FeatureCollection('users/rmohon835/NW_border').geometry()
pheno=ee.ImageCollection("MODIS/006/MCD12Q2")
cornArea=ee.Image('projects/ee-rmohon835/assets/cornArea/NDVImax2005')

ls=['Greenup_1','MidGreenup_1','Peak_1','Maturity_1','Senescence_1','MidGreendown_1','Dormancy_1']

for year in range(2004,2020):
  cornArea=ee.Image('projects/ee-rmohon835/assets/cornArea/NDVImax'+str(year))
  phenoData=pheno.filterBounds(border).filterDate(str(year)).first().clip(border)
  for i in range(len(ls)):
    stageName=ls[i]
    outName=str(year)+stageName
    image=phenoData.select(stageName).mask(cornArea).clip(border)
    task=ee.batch.Export.image.toDrive(
      image=image,
      description=outName,
      folder='PhenoAllYears',
      region=border,
      crs='EPSG:4326',
      scale=1000,
      maxPixels=int(1e13))
    task.start()
    print(outName)

In [ ]:
# Export Two-Stage-Blended Pheno data on 0907
from IPython.display import Image
import pprint
# a2005 = ee.Image("projects/ee-rmohon835/assets/cornArea/NDVImax2005")
border = ee.FeatureCollection("projects/ee-rmohon835/assets/cornArea/NW_border").geometry();

# Reference images for sanp and intersect (namely, NDVImax2005-2019)
refImageYears=[]
for i in range(2005,2020):
  img = ee.Image("projects/ee-rmohon835/assets/cornArea/NDVImax"+str(i))
  refImageYears.append(img)
# print(len(refImageYears))

# MODIS pheno data
ModPheno=ee.ImageCollection("MODIS/006/MCD12Q2")

# Date function
def DateToUnix(date):
  return ee.Number(ee.Date(date).millis()).divide(1000*3600*24).getInfo();

# print(DateToUnix('2005-01-01'))
stages=['Greenup','MidGreenup','Peak','Maturity',
  'Senescence','MidGreendown','Dormancy']
# thisStage=stages[i] 
i=0
for year in range(2005,2020):

  # #  step1: create mask for cycle 1 using gp1
  pheno=ModPheno.filterDate(str(year),str(year+1)).filterBounds(border);

  greenUp1=pheno.select('Greenup_1').first().clip(border);  #  all greenUp1 data labeled 2005
  greenUp1In2005=greenUp1.gt(DateToUnix(str(year)+'-01-01'));     #  greenUp1 data in 2005
  greenUp1In2005Mask=greenUp1In2005.selfMask();             #  greenUp1 mask in 2005
  
  dorm1=pheno.select('Dormancy_1').first().clip(border);    #  all dorm1 data labeled 2005
  dorm1In2005=dorm1.lt(DateToUnix(str(year)+'-12-31'));           #  dorm1 data in 2005
  dorm1In2005Mask=dorm1In2005.selfMask();                   #  dorm1 mask in 2005

  c1BothIn2005Mask=greenUp1In2005Mask.And(dorm1In2005Mask); #  Both ends lie in 2005 for cycle 1



  # #  step2: create mask for cycle 2 using gp2

  greenUp2=pheno.select('Greenup_2').first().clip(border);  #  all greenUp2 data labeled 2005
  greenUp2In2005=greenUp2.gt(DateToUnix(str(year)+'-01-01'));     #  greenUp2 data in 2005
  greenUp2In2005Mask=greenUp2In2005.selfMask();             #  greenUp2 mask in 2005

  dorm2=pheno.select('Dormancy_2').first().clip(border);    #  all dorm2 data labeled 2005
  dorm2In2005=dorm2.lt(DateToUnix(str(year)+'-12-31'));           #  dorm2 data in 2005
  dorm2In2005Mask=dorm2In2005.selfMask();                   #  dorm2 mask in 2005

  notInC1Both=c1BothIn2005Mask.unmask().Not().clip(border).selfMask();
  c2BothIn2005Mask=greenUp2In2005Mask.And(dorm2In2005Mask).And(notInC1Both); #  Both ends lie in 2005 for cycle 2

  for stage in stages:

    # change by selection of stage
    stage1AsOutput=pheno.select(stage+'_1').first().clip(border);
    Stage1BothIn2005=stage1AsOutput.updateMask(c1BothIn2005Mask); #  GreenUp of Both ends lie in 2005 for cycle 1

    # change by selection of stage
    Stage2AsOutput=pheno.select(stage+'_2').first().clip(border);  #  all greenUp2 data labeled 2005
    Stage2BothIn2005=Stage2AsOutput.updateMask(c2BothIn2005Mask).clip(border); #  GreenUp of Both ends lie in 2005 for cycle 2

    # #  step3: blend cycle 1 and 2
    ThisYearStage=Stage1BothIn2005.blend(Stage2BothIn2005);
    ThisYearStage=ThisYearStage.rename(['pheno']);

    # #  test if time range is suitable, and cut the 1% tail 
    stats=ThisYearStage.reduceRegion(
      reducer=ee.Reducer.percentile(percentiles=[0.5,99.5],
        outputNames=['p05','p995']),
      geometry=border,
      scale=1000
    )

    pct005=ee.Number(stats.get('pheno_p05')).toInt().getInfo()
    pct995=ee.Number(stats.get('pheno_p995')).toInt().getInfo()
    # print(pct005)
    # print(pct995)
    StageThisYear=ThisYearStage.updateMask(
      ThisYearStage.gte(pct005).And(ThisYearStage.lte(pct995))
      );
    StageThisYear=StageThisYear.mask(refImageYears[i]).clip(border)
    
    # url=StageThisYear.reproject('EPSG:4326').getThumbUrl({
    #     'min':12700,'max':13100,
    #     'dimensions':1000,
    #     'region':border,
    #     'palette':['green','red']
    #     })
    

    task=ee.batch.Export.image.toDrive(
      image=StageThisYear,
      description=str(i+2005)+'_'+stage,
      folder='pheno_colab_0907',
      region=border,
      crs='EPSG:4326',
      scale=1000
    )
    task.start()
    # pprint.pprint(refImageYears[i].getInfo())
    print(str(i+2005)+'_'+stage)
  i+=1


# Image(url=url)

In [ ]:
# export era5 temperature and precipitation data
from IPython.display import Image
border = ee.FeatureCollection("projects/ee-rmohon835/assets/cornArea/NW_border").geometry();
# era5day=ee.ImageCollection("ECMWF/ERA5/DAILY").filterBounds(border).filterDate('2019-07-25','2020');

# imgLs=era5day.toList(era5day.size())
# collectionSize=imgLs.size().getInfo()
# print(era5day.size().getInfo())



# GEE

data = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY");
border = ee.FeatureCollection("users/rmohon835/NW_border").geometry();

oneDayData=data.filterDate('2005-01-01','2005-01-02').select('surface_net_solar_radiation');
# print(oneDayData);

aggregatedDay=oneDayData.sum().clip(border);

# print(aggregatedDay.propertyNames());
# print(aggregatedDay.get('system:index'));


# for each year

for year in range(2005,2020):
  onePeriodData=data.filterDate(str(year)+'-01-01',str(year)+'-02-01').select('surface_net_solar_radiation');  
  size=oneDayData.size().getInfo()
  cnt=1
  for i in range(1,365+1,8):
    if cnt>=3:
      break
    oneDayData=onePeriodData.filter(ee.Filter.calendarRange(i,i+7,'day_of_year'))
    sizeCollection=oneDayData.size().getInfo()
    print(sizeCollection)
    imgLs=oneDayData.toList(sizeCollection)
    # print(oneDayData.first().get('system:index').getInfo())
    outNSSR=oneDayData.sum().clip(border)
    date1=oneDayData.first().get('system:index').getInfo()[:-3]
    size2=imgLs.size().getInfo()
    print(size2)
    date2=ee.Image(imgLs.get(size2-1)).get('system:index').getInfo()[:-3]
    
    print(date1,date2)
    cnt+=1
    # task=ee.batch.Export.image.toDrive(
    #   image=outNSSR,
    #   description=date++'NSSR',
    #   folder='NSSR_0915',
    #   region=border,
    #   crs='EPSG:4326',
    #   scale=11000
    # )
    # task.start()
    break 

  break
 

192
192
20050101 20050108
